<a href="https://colab.research.google.com/github/KelianF/BioInformatics/blob/master/Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
def Count(Motifs):
    count = {} # initializing the count dictionary
    k = len(Motifs[0])
    for symbol in "ACGT":
        count[symbol] = []
        for j in range(k):
            count[symbol].append(0)

    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

def Consensus(Motifs):
    k = len(Motifs[0])
    count = Count(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

def Score(Motifs):
    consensus = Consensus(Motifs)
    t = len(Motifs)
    k = len(Motifs[0])
    score = 0
    for i in range(k):
        FrequentSymbol = consensus[i]
        for j in range(t):
            if Motifs[j][i] != FrequentSymbol:
                score = score + 1
    return score

def Pr(Text, Profile):
    p = 1
    for i in range(len(Text)):
        p = p * Profile[Text[i]][i]
    return p

def ProfileMostProbablePattern(Text, k, Profile):
    p_dict = {}
    for i in range(len(Text)- k +1):
        p = Pr(Text[i: i+k], Profile)
        p_dict[i] = p
    m = max(p_dict.values())
    keys = [k for k,v in p_dict.items() if v == m]
    ind = keys[0]
    return Text[ind: ind +k]

def CountWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    count = {}
    for symbol in "ACGT":
        count[symbol] = []
        for j in range(k):
            count[symbol].append(1)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    count = CountWithPseudocounts(Motifs)
    profile= {}
    ## divide the number of motif strings to get frequency
    for letter in count.keys():
        profile[letter] = [float(x)/ (t+4) for x in count[letter]]
    return profile


def GreedyMotifSearchWithPseudocounts(Dna, k, t):
    BestMotifs = []
    for i in range(0, t):
        BestMotifs.append(Dna[i][0:k])
    n = len(Dna[0])
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            P = ProfileWithPseudocounts(Motifs[0:j])
            Motifs.append(ProfileMostProbablePattern(Dna[j], k, P))
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
    return ' '.join([x for x in BestMotifs])

In [29]:
k=15
t=20
DNA = [
'ATTGCCATAGAACGAGCAGCTACGGACGGCCCGCATTCAACACTCCTAAAACGTGGCTACCGAACGCTGTCCCACGTGTTACCCCGTTGTGGATCCCTGCCCAAGCTATGACCAATACTCTGTTATCTATGGATGAGAAACTTCTAACGATCATCTTCCCCATGCGCGGTCCGAATTGCCATAGAACGA',
'GCAGCTACGGACGGCCCGCATTCAACACTCCTAAAACGTGGCTACCGAACGCTGTCCCACGTGTTACCCCGTTGTGGATCCCTGCCCAAGCTATGACCAATACTCTGTTATCTATGGATGAGAAACGTGCACACCCTAGTCTTCTAACGATCATCTTCCCCATGCGCGGTCCGAATTGCCATAGAACGA',
'CCATACCCAGCAGCCTAGCGTCACTCCGGCACAAACGTGTAGATATTCCGATGATGGGGGTGGCGACCCTGCCCAGAATTTGGAGACGAAATACGAATAGGCTGCGGGTGAGTTGAATAACAGATTAGACACTTTGGCAGGTCATACTGTGCGTTAGAGGGGAACGGCCCCTACTTGGGCACGCGTTAC',
'TCTTAATCGACTCTTGAGCGCCGGGTATAACGAATGTGAGTCCAGACGAATGGTCGAGCTTCTTCTGATGTTCAGACCCTAGTCGACTACTTTTGGATGAGATAAGCCCCCCGGATCGGGGGTGGCCATGCACAGGAGCGGGCCGACGCTGTCTCGCAATTCATCCCTATTCCCGTTCTTTCCCCAGCG',
'CCACCTCGCACACACTGGCAGATGCCGAGTTGAGGTTTAGGCACAAAGGTCATGAACTACCAGCAATGCGTGTCCTGTCTTATTGAACGGTACGTGGCGACCCGGCTCTAAACTATCAGCCCGGTTTTACCCAGCTATTGGCGACGGTTTACACGGATATTTTCACGGGGGTCGCGAGCAATATGCGAG',
'ACAAGAATCCCGGGCAAGCGTCGCAGTTAGATGATTTGTCCTAGCTACGATGGCGACCCTAGCATTTATCCTGAGCTGTGAAGAAGATGAAGGTACTTGTTAGGGTGGTCCAGGGCGGTGGGAGGGGGAAGGCGATCTGCGATCCTCGCATCCATATGTCTAGATGAAGGCGCATCAAGTCCCTCACAA',
'ATGTGAGATATGGCCTTCTGGGCAGACACCACACACACGGTAACTGGATGACCCCTCGGCTGCTGGTGGCGAAATTAGTCAGTTGCTAGGGAGTGGGAGAGGAGGAGCGTGTGACGAGGAATGTGTGGTCTTGAAACGAAGCGTGCCTCGCACTTTTTTTCTGCAATATTTCCCTCGATTTAAGGTCCA',
'AAACTGCGATGAAGTGCAAGGCGTACATAGACCGCTACAGTCTTGCTGTCTTAGCCCTATGAACCAATGCCCCAGTAAGGGATCGGCCCTTGCCCGCCCAAGGTGAATTAAAGCGTGGCGCATCTAGTCGCGCAGGAGCATGCAGGAATTGAGTAAACATCTTCTATGTTTAGATCCCGTTCAGACGCC',
'ACTCATCACCTACTGCTTAGTCATATAGATGATACGATACGGCGAGCCTCGCAGTCCCTGGGTGGCATACCTAGTCATGGTCCCACCTCAGGTGTGGAGTATCTGCTGTTTTTAGTACCAGGATTGAATGAAGCCTAGGGCCTCTGTATCTGAATTTTACCGTTGACTGCGACTCCGGAGGGTCCCGGG',
'TCGCTCGTGGTGACTGATGTCCACTTCGTAGGGTACTATTAACCCGTGGCTGACCTAGTCCAGCCTACATAGGACGATAGGAATAACGCTGGCTTCGTTACCCACGCGAAATATCAAAGACGCCAAAACACGGAGGGGTCCCAGTGGGCTGATAGTTTCTAGGCCGCGGTTAACCCGTTAGTCTCGTTG',
'GCTCAAGAAGTGAAAACCTGCGCTGCACATGACGATGACAGACCAGGAGTACGAGTCACTCACAATCCTTCAAAAACAGTCACTCGTGTGATCCGACCCTAGTCCCTCTTCATGTGACTGACAAAGACCGCTATCTTTGAAGAACCAATGTGTTATTGCCAAGTCGGAAATGTTAGAGCACCAGTTGGG',
'AGGTACCGTGCATTGTCATCTTTTGCCTCTATTCTTCACGAGGGTCCGTACCATATGACCTTTTATCCATAGAGATTACGGGCCATAATCTATGGTGATTTGGTAGGACCCCGTAGTCAACAACACACGTTTGAACACAGAGAGTGGCGACCTGTGTCCACAAAGTAGGTACGTTGCGCTGCAGGAAGT',
'GTTCGAAGACTAACCCAGGGCCCACTATACAATAGTTACAAGCTGGCCCTAACGATCTTAATCGTAAACCTCCACGACATATACATTAAGGATATATGTTGAATCGTGGTCTCAGTGGGCACTAAGAGCTAGTTAAGAGGTGGGATCCCTAGTCCGATCCTCAGGCGAGCGCGTCGTGACTGCTAGTAA',
'GATAGTTGTGATCACCCTAGTCCGTTTTTAGCGCCGTACGTGACGATTATCGCGAGGAATCACACCTTTATCTCCCTAGAGTCAATGGCCCGATTCGATATAGTATCGGCTAGTGCCTTCATAAGTAACCACTCAGTTCAGCTCTAGCAATTACAAGGACCACGCCTCTGTCTGAGCAATGACACCGAA',
'CTGTAATAGTTTCCGCATATTCCGATCCGAAGAGTCGAGGATTATCCTTGACCACGGGTGCAGAGTGGTCGCCGGTGCCTGTATGTAGATCCTGCAATAGTGGGCCCCCTAGTCGAATTACACTGTATCGCCGCCGGCGGTTTAACCTAATTCCTTTACTGATGCTTTTATTATAACGTTGTCCCGAGG',
'ACATTGCATAATGCTACCTGGGCGTAGGTGGCGGAACTAGTCATATCCTAACCGACCCTGCGAGAGCACCTTTTGCCTTCCATGTCATTCGGAATGCGGTGACGTATCCACGAATGTCTGAGATGAATAGGCTAAGGCGGTGCATTACGAAAGTTGAAATGAAGACAACTAGACGCACTATTTGCGATT',
'TTTTCGCCAATCCAGTTATGTGGTGCTCTCTACAGCCGGCAAGTCCCTGAGCCAGACTTGAATTTCTGTGGGCGACCCTAGTTGCGTACCGATCCTAAGTAGTCTGAGCACGAGGAAGTTGTGGTGTCTCCGGTGATCACCTTACCGGCTTGATTAGCTCTGCCTCAGGTCCATCGATTTTACTGCATA',
'TAGGGTCAAGTGCGTTGAACCGGGTGAACGCGTCCGTTCTGAACCATTTCCGATTTGCGGCGAATCTTCGGACAGATAAAATGGCAGAGGGCGACTCATGGCCAATGTTAACGGTACCTGTGTACCTAGAGGTCAGCATGAGGAATTCGACGTGGCTACCCCGGCAACCGAAGTGGCGACCCTAAGGTA',
'TCAAATAAGCCTCGTGATGTAGTCGCTTCATGAAGAGTGCGAGTAAGCCCCCATGAGTAGGTGGACCGGAGAGCGAACTCAAGTATTTAACGTTTAACGCTATCCACGTAGCTTGGTCAGCGCCCCGCGACCCTAGTCCCTGAGGAGGGTAGGGGGCGTCATCCTTAGGTGAGCTATCACCCATAGTCG',
'CGTACGTGGGATACTGATGGCATGCTGGGCAAACTCATCCCAAAACATGCAATGCTTCGTGCCATGCAGGGGGCGAACCTGTGGCGACGAAAGTCTATGCCCCTCTGGCAAGACAACGTGATTAACCCTTCACCACCTGGGCCGCTCTTCACATGCGATGGGCGCTATAGGACCTTCAACAGTGGCTAA']

GreedyMotifSearchWithPseudocounts(DNA, k, t)

'GTGGCTACCGAACGC GTGGCTACCGAACGC GTGGCGACCCTGCCC GGAGCGGGCCGACGC GTGGCGACCCGGCTC ATGGCGACCCTAGCA GTGGCGAAATTAGTC GTGGCGCATCTAGTC GTGGCATACCTAGTC GTGGCTGACCTAGTC GATCCGACCCTAGTC GTGGCGACCTGTGTC GTGGGATCCCTAGTC GTGATCACCCTAGTC GTGGGCCCCCTAGTC GTGGCGGAACTAGTC TGGGCGACCCTAGTT GTGGCGACCCTAAGG CCCGCGACCCTAGTC GTGGCGACGAAAGTC'

In [31]:
k=3
t=4
DNA = [
'TGACGTTC',
'TAAGAGTT',
'GGACGAAA',
'CTGTTCGC'
]

GreedyMotifSearchWithPseudocounts(DNA, k, t)

'ACG AAG ACG TCG'